In [ ]:
!pip3 install mesa

In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `MultiGrid` que fuerza con uno o multiples objetos por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [3]:
def get_grid(model):
  grid = np.zeros((model.grid.width, model.grid.height))
  for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    for content in cell_content:
      if isinstance(content, CleaningAgent):
        grid[x][y] = 2
      else:
        grid[x][y] = content.live
  return grid

In [36]:
class GameCleanOrDirtyAgent(Agent):
  
  def __init__(self, unique_id, model, CorD):
    super().__init__(unique_id, model)
    ##self.live = np.random.choice([0,1])
    self.live = CorD
    if self.live == 1:
      model.addDirtyTiles()
    self.next_state = None
    self.cleaned = False;

  def step(self):
    self.next_state = self.live

  def advance(self):
    self.live = self.next_state
    self.cleaned = False;



In [5]:
class CleaningAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.next_state = None

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
        self.pos,
        moore = True, #Allows agent to be able to go from top of grid to bottom, and right side to left side (vice versa)
        include_center = False #Prevents agent from staying in the same spot
    )
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)

  def determineType(self):
    cellmates = self.model.grid.get_cell_list_contents([self.pos])
    for i in cellmates:
      if type(i) == GameCleanOrDirtyAgent:
        if i.live == 1:
          if i.cleaned == False:
            i.cleaned = True
            i.next_state = 0
            model.addCleanTile()
        else:
          self.move()

  def step(self):
    self.determineType()
    


In [94]:
class GameCleanModel(Model):

  def __init__(self, width, height, totalCleaners, percentageDirtyTiles):
    self.num_agents = width * height
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.numStepsDone = 0
    self.numTilesCleaned = 0
    self.numDirtyTiles = 0
    self.totalDirtyTiles = float(int(self.num_agents * (percentageDirtyTiles/100)))
    self.totalCleanTiles = self.num_agents - self.totalDirtyTiles
    print(self.totalDirtyTiles)
    print(self.totalCleanTiles)

    #Create agents
    for (content, x, y) in self.grid.coord_iter():
      CorD = np.random.choice([0,1])
      if CorD == 0:
        if self.totalCleanTiles > 0:
          a = GameCleanOrDirtyAgent((x, y), self, 0)
          self.totalCleanTiles -= 1
        else:
          a = GameCleanOrDirtyAgent((x, y), self, 1)
          self.totalDirtyTiles -= 1
      if CorD == 1:
        if self.totalDirtyTiles > 0:
          a = GameCleanOrDirtyAgent((x, y), self, 1)
          self.totalDirtyTiles -= 1
        else:
          a = GameCleanOrDirtyAgent((x, y), self, 0)
          self.totalCleanTiles -= 1

      self.grid.place_agent(a, (x, y))
      self.schedule.add(a)

    #Create agent cleaners
    for i in range(totalCleaners):
      a = CleaningAgent(i, self)
      self.schedule.add(a)
      self.grid.place_agent(a, (1, 1))

    self.datacollector = DataCollector(model_reporters = {"Grid": get_grid})

  def addCleanTile(self):
    self.numTilesCleaned += 1

  def addDirtyTiles(self):
    self.numDirtyTiles += 1


  def step(self):

    self.datacollector.collect(self)
    self.schedule.step()
    self.numStepsDone += 1

In [187]:
# Definimos el tamaño del Grid
GRID_SIZE = 10

#Definimos la cantidad de AgentCleaners
TOTAL_CLEANERS = 20

#Definimos el porcentaje de celdas sucias
CELDAS_SUCIAS = 60

# Definimos el tiempo a correr
TIEMPO_MAX = 5

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = GameCleanModel(GRID_SIZE, GRID_SIZE, TOTAL_CLEANERS, CELDAS_SUCIAS)
while (time.time() - start_time) < TIEMPO_MAX:
    if (model.numDirtyTiles == model.numTilesCleaned):
      break;
    model.step()

NUM_GENERATIONS = model.numStepsDone
celdasLimpias = (((GRID_SIZE * GRID_SIZE) - (model.numDirtyTiles - model.numTilesCleaned)) / (GRID_SIZE * GRID_SIZE)) * 100
print('Tiempo necesario hasta que todas las celdas estén limpias:', str(datetime.timedelta(seconds=(time.time() - start_time))))
print("Porcentaje de celdas limpias después del termino de la simulación: " + str(celdasLimpias) + "%")
print("Número de movimientos realizados por todos los agentes: " + str(model.numStepsDone))

60.0
40.0
Tiempo necesario hasta que todas las celdas estén limpias: 0:00:00.030174
Porcentaje de celdas limpias después del termino de la simulación: 100.0%
Número de movimientos realizados por todos los agentes: 43


In [122]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [123]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [124]:
anim